In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Jeju_dacon/train_data.csv')
trade = pd.read_csv('/content/drive/MyDrive/Jeju_dacon/trade.csv')

In [38]:
df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),y-m,holiday
0,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01,0
1,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019-01,0
2,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019-01,1
3,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0,2019-01,0
4,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0,2019-01,0
...,...,...,...,...,...,...,...,...,...
23937,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023-02,0
23938,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023-02,0
23939,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023-03,1
23940,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023-03,0


In [20]:
test = pd.read_csv('/content/drive/MyDrive/Jeju_dacon/test.csv')

In [5]:
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7922 sha256=24e886195276cebfb7b33c98b505c6a338d20678df8de41a83561d4f092f6680
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [21]:
from pytimekr import pytimekr

year_2019 = pytimekr.holidays(year=2019)
year_2020 = pytimekr.holidays(year=2020)
year_2021 = pytimekr.holidays(year=2021)
year_2022 = pytimekr.holidays(year=2022)
year_2023 = pytimekr.holidays(year=2023)



def holidays(x):
    if x.weekday() in range(5,8):
        return 1
    if x.year == 2019  and x in year_2019 :
        return 1
    elif x.year == 2020 and x in year_2020:
        return 1
    elif x.year == 2021 and x in year_2021 :
        return 1
    elif x.year == 2022 and x in year_2022 :
        return 1
    elif x.year == 2023 and x in year_2023:
        return 1
    else:
        return 0

In [39]:
test['timestamp'] = pd.to_datetime(test['timestamp'])

In [40]:
import warnings
warnings.filterwarnings('ignore')
test['holiday'] = test['timestamp'].apply(holidays)

In [41]:
test['y-m']=test['timestamp'].dt.strftime('%Y-%m')

In [42]:
test

,ID,timestamp,item,corporation,location,holiday,y-m
0,TG_A_J_20230304,2023-03-04,TG,A,J,1,2023-03
1,TG_A_J_20230305,2023-03-05,TG,A,J,1,2023-03
2,TG_A_J_20230306,2023-03-06,TG,A,J,0,2023-03
3,TG_A_J_20230307,2023-03-07,TG,A,J,0,2023-03
4,TG_A_J_20230308,2023-03-08,TG,A,J,0,2023-03
...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,0,2023-03
1088,RD_F_J_20230328,2023-03-28,RD,F,J,0,2023-03
1089,RD_F_J_20230329,2023-03-29,RD,F,J,0,2023-03
1090,RD_F_J_20230330,2023-03-30,RD,F,J,0,2023-03


In [43]:
df.drop(['supply(kg)'], axis=1, inplace=True)

In [30]:
df.drop(['timestamp'], axis=1, inplace=True)

In [13]:
test.drop(['timestamp'], axis=1, inplace=True)

# Linear Regression

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 'ID' 컬럼을 해시 인코딩
df['ID'] = df['ID'].apply(lambda x: hash(x) % (10 ** 8))  # 임의의 큰 수로 나눔

# 'item' 컬럼을 레이블 인코딩
label_encoder_item = LabelEncoder()
df['item'] = label_encoder_item.fit_transform(df['item'])

# 'corporation' 컬럼을 One-Hot 인코딩
df = pd.get_dummies(df, columns=['corporation'], drop_first=True)

# 'location' 컬럼을 이진 인코딩
df['location'] = df['location'].apply(lambda x: 1 if x == 'loc2' else 0)


df['y-m'] = pd.to_datetime(df['y-m']).dt.month

# 설명변수와 예측변수 분리
X = df.drop('price(원/kg)', axis=1)
y = df['price(원/kg)']

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 4606901.214922261
R-squared: 0.17139231956090795


In [15]:
y_pred

array([2780.61913894, 2491.33851158, 4482.12252857, ..., 1867.80231696,
       2883.704712  , 4545.40801818])

In [16]:
df.isnull().sum()

ID               0
item             0
location         0
price(원/kg)      0
y-m              0
holiday          0
corporation_B    0
corporation_C    0
corporation_D    0
corporation_E    0
corporation_F    0
dtype: int64

# XGBoost

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 'y-m' 컬럼을 월로 변환
df['y-m'] = pd.to_datetime(df['y-m']).dt.month

# 설명변수와 예측변수 분리
X = df.drop('price(원/kg)', axis=1)
y = df['price(원/kg)']

# 범주형 변수를 더미 변수로 변환
X = pd.get_dummies(X, columns=['ID', 'item', 'corporation', 'location'], drop_first=True)

# test['y-m'] = pd.to_datetime(test['y-m']).dt.month

# # 범주형 변수를 더미 변수로 변환
# x_test = pd.get_dummies(test, columns=['ID', 'item', 'corporation', 'location'], drop_first=True)

# # 데이터셋 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 회귀 모델 생성
model = XGBRegressor()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 3285130.1721210023
R-squared: 0.40912905120593257


# 시계열 모델

SARIMA vs Prophet

In [32]:
!pip install pystan==2.19.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystan: filename=pystan-2.19.1.1-cp310-cp310-linux_x86_64.whl size=61975951 sha256=a10a0eb61baec83ede18c175982832291aa8b47c619c51f7d7f44ff8f112462f
  Stored in directory: /root/.cache/pip/wheels/3d/1c/94/4516243362eedbedad15ac4389691ee3bf2d45bec2639c9d8b
Successfully built pystan


In [33]:
!apt-get install -y build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [34]:
!pip install --upgrade setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [35]:
!pip install fbprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 100.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=726aa2e196504cf8b88bc8686bfdcfdecdc5991a73fcab95018f6c5dcdab259e
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built p

In [36]:
!pip install prophet

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


df['timestamp'] = pd.to_datetime(df['timestamp'])

# 데이터셋 split
train_size = int(len(df) * 0.8)
train, test = df[0:train_size], df[train_size:]

# SARIMA 모델
sarima_model = SARIMAX(train['price(원/kg)'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_result = sarima_model.fit(disp=False)

# 예측 기간 설정
sarima_forecast = sarima_result.get_forecast(steps=len(test))
sarima_pred = sarima_forecast.predicted_mean

# 성능 평가
mse_sarima = mean_squared_error(test['price(원/kg)'], sarima_pred)
mae_sarima = mean_absolute_error(test['price(원/kg)'], sarima_pred)
rmse_sarima = np.sqrt(mse_sarima)
mape_sarima = np.mean(np.abs((test['price(원/kg)'] - sarima_pred) / test['price(원/kg)'])) * 100

print(f'SARIMA Mean Squared Error: {mse_sarima}')
print(f'SARIMA Mean Absolute Error: {mae_sarima}')
print(f'SARIMA Root Mean Squared Error: {rmse_sarima}')
print(f'SARIMA Mean Absolute Percentage Error: {mape_sarima}')